In [7]:
# Imports
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
from scipy.spatial import distance


# Read images
im_dictionary = {}

for i in range(0,4):
    im_dictionary["Image{0}".format(i)] = cv2.imread("Example_Pictures/IMG_"+str(8383+i)+".jpg")

In [9]:
# Detect background point

# Lower and upper boundary for the colors of the 
# background point
lower = (230,230,230)
upper = (255,255,255)

# All colors outside of the lower and upper boundary
# will be deleted
mask_dictionary = {}
for i in range(0,4):
    mask_dictionary["Mask{0}".format(i)] =  cv2.dilate(cv2.inRange(im_dictionary["Image{0}".format(i)], lower, upper), None, iterations=1)



# Setup SimpleBlobDetector parameters
params = cv2.SimpleBlobDetector_Params()


# Change thresholds
params.minThreshold = 0;
params.maxThreshold = 24;

# Delete detected blobs that are too small or too
params.filterByArea = True
params.minArea = 3500
params.maxArea = 400000


# Detect blobs.
blob = cv2.SimpleBlobDetector_create(params)


keypoint_dictionary = {}
for i in range(0,4):
    keypoint_dictionary["Keypoints{0}".format(i)] = blob.detect(255-mask_dictionary["Mask{0}".format(i)])

In [10]:
# Save the coordinates of the detected blobs/keypoints
background_coordinates_dictionary = {}
for i in range(0,4):
    background_coordinates_dictionary["Coordinates{0}".format(i)] =  [keypoint.pt for keypoint in keypoint_dictionary["Keypoints{0}".format(i)]]

In [11]:
# Detect the laserpoint
lower = (190,110,130)
upper = (255,255,255)

mask_dictionary = {}
for i in range(0,4):
    mask_dictionary["Mask{0}".format(i)] = cv2.dilate(cv2.inRange(im_dictionary["Image{0}".format(i)], lower, upper), None, iterations=1)


# Setup SimpleBlobDetector parameters
params = cv2.SimpleBlobDetector_Params()


# Change thresholds
params.minThreshold = 0;
params.maxThreshold = 24;


# Delete detected blobs that are too small or
# too large
params.filterByArea = True
params.minArea = 50
params.maxArea = 40000

# set the minimum convexity for the blobs (the default
# value is higher)
params.filterByConvexity = True
params.minConvexity = 0.38


# Detect blobs
blob = cv2.SimpleBlobDetector_create(params)



laserpoint_dictionary = {}
for i in range(0,4):
    laserpoint_dictionary["Keypoints{0}".format(i)] = blob.detect(255-mask_dictionary["Mask{0}".format(i)])


In [12]:
# Save coordinates of the laserpoint
laser_coordinates_dictionary = {}
for i in range(0,4):
    laser_coordinates_dictionary["Coordinates{0}".format(i)] =  [keypoint.pt for keypoint in laserpoint_dictionary["Keypoints{0}".format(i)]]

In [8]:
width = len(mask_dictionary["Mask6"][0])
height = len(mask_dictionary["Mask6"])

print("Images have a size of "+str(width)+" x "+str(height))

Images have a size of 3456 x 2304


In [14]:
"The size of the background point is 40mm."

'The size of the background point is 40mm.'

In [17]:
"The size of the background point is "+str(round([keypoint.size for keypoint in keypoint_dictionary["Keypoints0"]][0])
)+" pixels on our image."

'The size of the background point is 314 pixels on our image.'

In [18]:
factor = 40/round([keypoint.size for keypoint in keypoint_dictionary["Keypoints0"]][0])
"We scale down our movements by the factor "+str(factor)+"."

'We scale down our movements by the factor 0.12738853503184713.'

In [19]:
# Calculate distance between the laserpoint and the background point in 
# each image
distance_dictionary = {}

for i in range(0,4):
    distance_dictionary["Distances{0}".format(i)] = [factor*(laser_coordinates_dictionary["Coordinates{0}".format(i)][0][0]-background_coordinates_dictionary["Coordinates{0}".format(i)][0][0]), factor*(laser_coordinates_dictionary["Coordinates{0}".format(i)][0][1]-background_coordinates_dictionary["Coordinates{0}".format(i)][0][1])]

In [20]:
import math
final_dictionary = {}

# This dictionary contains the movement of the laserpoint between 
# two images
# The movement (in mm) is calculated based on the changes in the
# distance between the laserpoint and the background point
for i in range(1,4):
    if i != 7 and i != 8:
        final_dictionary["Movement{0}".format(i)] = [math.sqrt((distance_dictionary["Distances{0}".format(i)][0]-distance_dictionary["Distances{0}".format(i-1)][0])**2+(distance_dictionary["Distances{0}".format(i)][1]-distance_dictionary["Distances{0}".format(i-1)][1])**2)]

In [22]:
# printing out an example mask with keypoints
keypoints = np.concatenate((keypoint_dictionary["Keypoints0"], laserpoint_dictionary["Keypoints0"]))

In [23]:
# To interrupt this, press any key
reversed_mask = 255-mask_dictionary["Mask0"]

cv2.namedWindow("output", cv2.WINDOW_NORMAL)        # Create window with freedom of dimensions
img_with_keypoints = cv2.drawKeypoints(reversed_mask, keypoints, None,color=(0,255,0), flags =cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img_with_keypoints = cv2.resize(img_with_keypoints, (960, 540)) # Resize image

# Show blobs
cv2.imshow("Keypoints", img_with_keypoints)
cv2.waitKey(0)

3

In [24]:
# Export movements to Excel
import xlsxwriter

workbook = xlsxwriter.Workbook('Laser_Movement.xlsx')
worksheet = workbook.add_worksheet()


row = 0

for col, data in enumerate(final_dictionary.values()):
    worksheet.write_column(0, col, data)

workbook.close()